# BigARTM CLI


## Приложение `bigartm.exe`

Приложение `bigartm.exe` (в Unix — просто `bigartm`) — самостоятельный бинарик, работающий по следующему алгоритму:

1. Загрузить словарь, если он указан
2. Инициализировать модель
3. Если батчи еще не созданы — прочитать корпус в указанном формате и создать батчи
4. Сделать определенное число итераций Online-EM алгоритма по указанным батчам, в соответствии с настройками
5. При последнем проходе по корпусу, вывести в файл предсказанные тематические профили документов (вектора $\theta$)
6. Вывести модель/словарь в файлы

#### Самостоятельный бинарник

Бинарник не должен зависеть от динамически линкуемых библиотек. При переносе бинарника на другую машину он должен запускаться без дополнительных файлов и переменных окружения. Самостоятельность бинарика значительно упростит его использование на кластере для распределенного обучения модели и вывода тематических профилей документов. 

Для сборки самостоятельного бинарика `gcc` нужно передавать флаг `-static`. В `cmake` нужно прописать что-то такое:
```
SET(CMAKE_EXE_LINKER_FLAGS "-static")
```

## Ключи консольного приложения

- `--rand-seed <seed>`: инициализация генератора случайных чисел, гарантируется что приложение отрабатывает с одинаковым результатом при одинаковом значении seed

### Dictionary

Использование существующего словаря:
- `--use-dictionary <dictionary-file>`: использовать словарь, загруженный из файла в бинарном формате
- `--use-dictionary-bow <vocab-txt>`: использовать текстовый словарь в формате vocab.xxx.txt из UCI Bag-of-Words
- настройки динамического словаря (в случае если он меняется), мин частота, макс число токенов

Если не указана опция использования существующего словаря, то по-умолчанию создается новый словарь.

Создание нового словаря по данным (токены добавляются автоматически как в [примере](https://github.com/bigartm/bigartm/blob/master/src/python/examples/example14_initialize_topic_model.py)):
- `--dictionary-min-items <N/P>`: брать только слова, которые присутствуют не менее чем в N документах (если указано целое >= 1) или P% документов (если указано вещественное < 1)
- `--dictionary-max-items <N/P>`: ... не более чем в N документах
- `--dictionary-min-total-count <N>`: брать только слова, которые встретились не менее N раз во всей коллекции

При построении словаря учитываются опции `use-modalities / ignore-modalities` (из Learning Options): словарь по неиспользуемым модальностям не составляется.

### Corpus

- `--corpus-format [vw,bow,mm]`: формат в котором потупает корпус текстов
- `--read-corpus <file/url>`: источник, из которого поступает корпус в сыром виде: stdin / файл / каталог-с-файлами / сеть
- `--read-batches <batches-path>`: путь к батчам, использовать заранее подготовленные батчи

По умолчанию считается что подготовленные батчи отсутствуют и сырой корпус читается из stdin в формате vw.

### Model

- `--load-model <model-file>`: загрузить модель из файла

Если не указана опция загрузки модели, то модель инициализируется.

- `--initialize <scheme>`: создать новую модель, инициализировав ее одной из доступных схем 
- `--num-topics`: задать фиксированное число тем
- `--topics <topics-file>`: указать файл с названиями топиков

### Learning Options

Базовые ключи:
- `--passes <N>`: число проходов по корпусу, по умолчанию N=1

Параметры обучения:
- `--inner-iterations-count <N>`
- `--decay-weight <w>`
- `--apply-weight <w>`

Модальности:
- `--use-modalities <modalities>`: использовать только указанные модальности
- `--ignore-modalities <modalities>`: отключить указанные модальности (используется одно из двух: use-modalities либо ignore-modalities)
- `--modality-weights <weights>`: назначить веса различным модальностям
- `--modality-weights word:1,author:10.0,link:0.2`

Регуляризация:
- регуляризатор задается набором `(тип, параметры, вес, модальности, топики)`
- модальности указываются в формате `@name`
- топики указываются в формате `#topic`
- `--regularizer smoothPhi:0.1`
- `--regularizer sparseTheta:25.0`
- `--regularizer "decorrelation:10 @word"`
- `--regularizer "specific(x,y):0.1 #topic1:0.1,topic2:2.0 @word:0.5,author:0.1"`

**Use case**: шумовые темы

**Идея**: конфигурационный файл с параметрами регуляризации вместо объемных параметров, хотя параметры можно тоже оставить. А может быть конфигурационный файл с вообще всеми параметрами обучения, аргументами задается только откуда вводить и куда выводить.

### Multicore Mode

- `--threads <N>`: число потоков в которых производить обучение, по умолчанию N=1

### Distributed Mode

*Пока не нужно реализовывать.*

### Output

- `--write-batches <batches-path>`: при первом чтении генерировать батчи и записывать по назначению
- `--write-model <model-file>`: по окончанию обучения записать модель в файл
- `--write-dictionary <dictionary-file>`: вывести словарь в файл
- `--write-predictions <predictions-file>`: вывести тематические профили документов с последнего прохода по коллекции, в формате CSV. Возможность вывода в stdout!
- `--write-model-readable <model-csv-file>`: по окончанию обучения записать модель в текстовый файл в формате CSV

## Информационный вывод и логирование

Информационный вывод должен производиться исключительно в stderr, поскольку в stdout может подаваться результат — тематические профили документов.

Что выводить в stderr:
- число проходов
- число обработанных документов
- метрики качества
  - перплексия
  - разреженность по модальностям
- способы подсчета метрик: hold-out / [progressive-validation](http://hunch.net/~jl/projects/prediction_bounds/thesis/mathml/thesisse44.xml) 
- логарифмический вывод (как в vw)

## Примеры использования

На этих примерах нужно протестировать новое CLI, а в конечном счете – сделать подробное описание этих примеров в документации.

#### 1. Эксперимент на датасетах UCI Bag-of-Words (`kos`, `nytimes`)

#### 2. Параллельное обучение модели мультиязычной Википедии

#### 3. Использование тематической модели Википедии для категоризации страниц в интернете

Продемонстрировать использование `bigartm` в Map-Reduce задаче при помощи стриминга.

## Пакетирование

### Пакет для Debian

В идеале BigARTM пользователь должен не собирать из исходников, а устанавливать вот таким образом:
`apt-get install bigartm`

Что нужно при этом установить:
- SO: `/usr/lib/bigartm.so`
- CLI: `/usr/bin/bigartm`
- Python-пакет: `/usr/lib/python2.7/site-packages`

[Руководство по созданию deb-пакетов](http://ubuntuforums.org/showthread.php?t=910717)